# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
!dir
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')


categories.csv			     Messages.db
DisasterResponse.db		     messagesDB.db
ETL\ Pipeline\ Preparation.ipynb     messagesDB.db.db
ETL\ Pipeline\ Preparation-zh.ipynb  ML\ Pipeline\ Preparation.ipynb
InsertDatabaseName.db		     ML\ Pipeline\ Preparation-zh.ipynb
messages.csv			     model.pkl
messages.db			     stopwords
messagesDB			     Twitter-sentiment-self-drive-DFE.csv
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///messagesDB.db')
df = pd.read_sql_table("messagesTab", engine)
X = df['message']
Y = df.drop(columns = ['id', 'message' , 'original' ,'genre'])
category_names = list(df.columns)



In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
#define tokenize function
def tokenize(text):
    #Convert to lowercase
    text = text.lower()
    #Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)    
    #Tokenize words
    tokens = word_tokenize(text)
    #stop words
    stop_words = stopwords.words("english")
    #normalize word tokens
    normalizer = PorterStemmer()
    normalized = [normalizer.stem(word) for word in tokens if word not in stop_words]
    return normalized

In [7]:
#testing tokenize function
tokenize(df["message"][0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [10]:
#train with pipeline
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [11]:
#predict all outputs
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [12]:
y_pred = pd.DataFrame(y_pred, columns = Y.columns)
y_pred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
def model_perform(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]),"\n\n")
    # print raw accuracy score 
    print('Accuracy Score: {}'.upper().format(np.mean(y_test.values == y_pred)),"\n\n")

In [14]:
model_perform(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.67      0.48      0.56      1241
          1       0.85      0.92      0.88      3963
          2       0.54      0.33      0.41        40

avg / total       0.80      0.81      0.80      5244
 


request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4325
          1       0.81      0.42      0.55       919

avg / total       0.87      0.88      0.87      5244
 


offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5221
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5244
 


aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3055
          1       0.75      0.59      0.66      2189

avg / total       0.75      0.75      0.74      5244
 


medical_help
             precision    re

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
#Get pipeline parameters
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f31e84011e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [16]:
#define selected parameters for gridsearch
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[4, 15], }


cv = GridSearchCV(pipeline, param_grid = parameters,verbose = 10, n_jobs=-1)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__min_df': [1, 5], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [4, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [17]:
#Fit the pipline with defined greadsearch parameters
cv.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__n_estimators=4, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=4, tfidf__use_idf=True, vect__min_df=1, score=0.2221427549706766, total=  37.3s
[CV] clf__estimator__n_estimators=4, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   53.6s remaining:    0.0s


[CV]  clf__estimator__n_estimators=4, tfidf__use_idf=True, vect__min_df=1, score=0.20483478758403662, total=  37.3s
[CV] clf__estimator__n_estimators=4, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=4, tfidf__use_idf=True, vect__min_df=1, score=0.21502145922746782, total=  37.1s
[CV] clf__estimator__n_estimators=4, tfidf__use_idf=True, vect__min_df=5 


...

[CV]  clf__estimator__n_estimators=4, tfidf__use_idf=False, vect__min_df=1, score=0.2234620886981402, total=  37.3s
[CV] clf__estimator__n_estimators=4, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  7.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=4, tfidf__use_idf=False, vect__min_df=5, score=0.22085538549563724, total=  33.3s
[CV] clf__estimator__n_estimators=4, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__n_estimators=4, tfidf__use_idf=False, vect__min_df=5, score=0.21670719496495494, total=  33.4s
[CV] clf__estimator__n_estimators=4, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__n_estimators=4, tfidf__use_idf=False, vect__min_df=5, score=0.22503576537911302, total=  33.2s
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1, score=0.24302674867687027, total= 1.3min
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1, score=0.22829352024030897, total= 1.3min
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=15, t

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 28.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__min_df': [1, 5], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [4, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
model_perform(cv, X_test, y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
#use another multioutputclassifier: AdaBoostClassifier

pipeline1 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [20]:
pipeline1.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f31e84011e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [21]:
#Train the second pipeline
pipeline1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [22]:
model_perform(pipeline1, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.67      0.12      0.20      1241
          1       0.78      0.98      0.87      3963
          2       0.36      0.12      0.19        40

avg / total       0.75      0.77      0.70      5244
 


request
             precision    recall  f1-score   support

          0       0.90      0.96      0.93      4325
          1       0.74      0.49      0.59       919

avg / total       0.87      0.88      0.87      5244
 


offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5221
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      5244
 


aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3055
          1       0.76      0.61      0.67      2189

avg / total       0.76      0.75      0.75      5244
 


medical_help
             precision    re

In [23]:
parameters1 = {
                'tfidf__use_idf': (True, False), 
                'clf__estimator__n_estimators': [50, 100],
                'clf__estimator__learning_rate': [2,3] 
}

In [24]:
#second cross validation
cv1 = GridSearchCV(pipeline1, param_grid=parameters1)
cv1

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__learning_rate': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
cv1.fit(X_train, y_train)

In [ ]:
model_perform(cv1, X_test, y_test)

### 9. Export your model as a pickle file

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv1, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.